- [ ] wie viel wir updaten 

- [ ] von was wir updaten 

- [ ] function um artikel: welcher artikel wurde um wie viel geupdated 

In [4]:
!pip install flask==2.3.3
from flask import Flask, request, jsonify
import pandas as pd
import smtplib
from email.mime.text import MIMEText
from email.mime.multipart import MIMEMultipart

app = Flask(__name__)

# Sample thresholds for medications
THRESHOLDS = {
    "Methadon": 2,
    "Dormicum": 1,
    "Bupaq": 5,
    "Esconarkon": 3,
    "Keta-S": 4,
    "Propofol": 2
}

# Load inventory
def load_inventory(file_path):
    """Load inventory from a CSV file."""
    try:
        return pd.read_csv(file_path, sep=',')
    except FileNotFoundError:
        raise FileNotFoundError(f"The file '{file_path}' does not exist.")
    except Exception as e:
        raise Exception(f"An error occurred while loading the inventory: {e}")

# Save inventory
def save_inventory(file_path, inventory):
    """Save inventory to a CSV file."""
    try:
        inventory.to_csv(file_path, index=False)
    except Exception as e:
        raise Exception(f"An error occurred while saving the inventory: {e}")

# Update inventory
def update_inventory(file_path, Name, quantity, operation):
    """
    Update the inventory for a specific product.

    Args:
        file_path (str): Path to the inventory file.
        Name (str): The ID of the product to update.
        quantity (int): The amount to add or remove.
        operation (str): The operation, either 'add' or 'remove'.

    Returns:
        pd.DataFrame: The updated inventory.
    """
    # Load the inventory
    inventory = load_inventory(file_path)
    
    # Check if the product exists
    print(f"before get inside------{inventory}")
    if Name in inventory['Name'].values:
        print("got inside")
        # Update quantity based on operation
        if operation == 'add':
            inventory.loc[inventory['Name'] == Name, 'quantity'] += quantity
        elif operation == 'remove':
            inventory.loc[inventory['Name'] == Name, 'quantity'] -= quantity
            # Ensure quantity doesn't drop below zero
            inventory.loc[inventory['Name'] == Name, 'quantity'] = \
                inventory.loc[inventory['Name'] == Name, 'quantity'].clip(lower=0)
        else:
            raise ValueError(f"Invalid operation '{operation}'. Supported operations: 'add', 'remove'.")
    else:
        # Raise an exception if the product ID is not found
        raise KeyError(f"Product Name '{Name}' not found in the inventory.")

    # Save the updated inventory
    save_inventory(file_path, inventory)
    return inventory

@app.route('/get_inventory', methods=['GET'])
def get_inventory():
    """API to retrieve the inventory as JSON."""
    try:
        # Get the file path from the query parameters
        file_path = request.args.get('file_path')
        
        if not file_path:
            return jsonify({'error': 'Missing file_path parameter'}), 400
        
        # Load the inventory
        inventory = load_inventory(file_path)
        return jsonify(inventory.to_dict(orient='records')), 200
    except FileNotFoundError as e:
        return jsonify({'error': str(e)}), 404
    except Exception as e:
        return jsonify({'error': str(e)}), 500


# Flask Routes
@app.route('/update_inventory', methods=['POST'])
def api_update_inventory():
    """API to update inventory."""
    try:
        data = request.json
        file_path = data['file_path']
        Name = data['Name']
        quantity = data['quantity']
        operation = data['operation']

        updated_inventory = update_inventory(file_path, Name, quantity, operation)
        return jsonify(updated_inventory.to_dict(orient='records')), 200
    except KeyError as e:
        return jsonify({'error': f'Missing key: {str(e)}'}), 400
    except ValueError as e:
        return jsonify({'error': str(e)}), 400
    except Exception as e:
        return jsonify({'error': str(e)}), 500

@app.route('/evaluate-medications', methods=['POST'])
def evaluate_medications():
    """API to evaluate medications and return the names below threshold."""
    try:
        data = request.get_json()
        if not data or 'file_path' not in data:
            return jsonify({'error': 'Missing file_path parameter'}), 400
        
        file_path = data['file_path']
        inventory = load_inventory(file_path)

        # Ensure inventory has the required columns
        if 'Name' not in inventory.columns or 'quantity' not in inventory.columns:
            return jsonify({'error': 'Invalid inventory format. Missing Name or quantity column.'}), 400

        # Filter medications below threshold
        below_threshold = inventory[
            inventory.apply(lambda row: row['Name'] in THRESHOLDS and row['quantity'] < THRESHOLDS[row['Name']], axis=1)
        ]['Name'].tolist()

        return jsonify({'below_threshold': below_threshold}), 200
    except Exception as e:
        return jsonify({'error': str(e)}), 500


# Configuration
SMTP_SERVER = "smtp.gmail.com"
PORT = 587
USERNAME = "vectorgroup209@gmail.com"
PASSWORD = "lydx inzx skaq qqgi"

def send_email(to_email, subject, body, from_email=None):
    try:
        msg = MIMEMultipart()
        msg['From'] = from_email if from_email else USERNAME
        msg['To'] = to_email
        msg['Subject'] = subject
        msg.attach(MIMEText(body, 'plain'))

        with smtplib.SMTP(SMTP_SERVER, PORT) as server:
            server.starttls()
            server.login(USERNAME, PASSWORD)
            server.sendmail(USERNAME, to_email, msg.as_string())

        return {"status": "success", "message": "Email sent successfully"}
    except Exception as e:
        return {"status": "error", "message": str(e)}

@app.route('/send-email', methods=['POST'])
def send_email_api():
    data = request.json
    to_email = data.get('to_email')
    subject = data.get('subject')
    body = data.get('body')
    from_email = data.get('from_email', USERNAME)

    if not to_email or not subject or not body:
        return jsonify({"status": "error", "message": "Missing required fields"}), 400

    result = send_email(to_email, subject, body, from_email)
    return jsonify(result)

# Run the Flask app
if __name__ == '__main__':
    app.run(host='0.0.0.0', port=8080)


  Using cached MarkupSafe-3.0.2-cp39-cp39-manylinux_2_17_x86_64.manylinux2014_x86_64.whl (20 kB)
  Attempting uninstall: MarkupSafe
    Found existing installation: MarkupSafe 2.0.0
    Not uninstalling markupsafe at /shared-libs/python3.9/py-core/lib/python3.9/site-packages, outside environment /root/venv
    Can't uninstall 'MarkupSafe'. No files were found to uninstall.

[notice] A new release of pip is available: 23.0.1 -> 24.3.1
[notice] To update, run: pip install --upgrade pip
 * Serving Flask app '__main__'
 * Debug mode: off
 * Running on all addresses (0.0.0.0)
 * Running on http://127.0.0.1:8080
 * Running on http://172.3.2.62:8080
Press CTRL+C to quit
172.3.50.42 - - [05/Dec/2024 19:29:44] "POST /send-email HTTP/1.1" 200 -


In [2]:
url -X POST   https://945152e0-66a8-4b9c-b068-364219c8e551.deepnoteproject.com/update_inventory   -H "Content-Type: application/json"   -d '{
        "file_path": "narcotics_bs.csv",
        "Name": "Methadon",
        "quantity": 50,
        "operation": "remove"
      }'

curl -X POST   https://bb57ba1c-1fe1-48ca-87e0-0530c677ee1b.deepnoteproject.com/send-email   -H "Content-Type: application/json"   -d '{
    "to_email": "turan.firat4@gmail.com",
    "subject": "Random Test Email",
    "body": "Hello Turan, this is a random test message sent via the Flask API." 
  }'


curl -X POST https://945152e0-66a8-4b9c-b068-364219c8e551.deepnoteproject.com/evaluate-medications \
-H "Content-Type: application/json" \
-d '{
  "file_path": "narcotics_ss.csv"
}'

SyntaxError: invalid syntax (3405718242.py, line 1)

<a style='text-decoration:none;line-height:16px;display:flex;color:#5B5B62;padding:10px;justify-content:end;' href='https://deepnote.com?utm_source=created-in-deepnote-cell&projectId=945152e0-66a8-4b9c-b068-364219c8e551' target="_blank">
 </img>
Created in <span style='font-weight:600;margin-left:4px;'>Deepnote</span></a>